<a href="https://colab.research.google.com/github/catalinux/pphw/blob/master/lda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pymc

import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import re
import pymc
import sys
from nltk.stem import PorterStemmer

nltk.download('punkt')
nltk.download('stopwords')

stemer = SnowballStemmer('english')
# prepare for cleaning data
stop_words = set(stopwords.words('english'))
# print(stop_words)
# reading data
read = pd.read_csv('data/dummy6.txt', header=None)
read.head()


# utility method for tokenize document
def tokenize(x):
    x = x.lower()
    s = re.sub(r'[^\w\s]', '', x)
    words = word_tokenize(s)
    tokens = [stemer.stem(w) for w in words if not w in stop_words]
    # tokenize = words
    return tokens


# now we have a data frame with words
df = read.applymap(tokenize)[0]

df.head()

vocab = np.unique(np.concatenate(df.values.flatten()))
# store position by word in a dictionary
dictionary = dict(zip(vocab, range(0, len(vocab))))
print(dictionary)


def encode(sentence):
    enc = np.zeros(len(sentence))
    for i, word in enumerate(sentence):
        enc[i] = dictionary[word]
    return enc.astype('int')


# encode our documents in position in vocab
df_encoded = np.empty(len(df), dtype=list)
for i, doc in enumerate(df):
    df_encoded[i] = encode(doc)

print(df_encoded)
# peek at a sentece
# print(vocab[df_encoded[8].astype(int)])

# count of topics
K = 2

# size of the dictionary
V = len(dictionary)

print("Dictionary size: ", V)
# parameter for per topic distributions
# phi word distribution for topic
# For each topic 𝑘, we draw its word distribution, which is denoted as 𝜑_𝑘.
# As prior for the per-topic word distribution we will use a V-dimensional symmetric Dirichlet distribution
beta = np.ones(V) / 10
phi = np.empty(K, dtype=object)
phi_temp = np.empty(K, dtype=object)
for i in range(K):
    phi_temp[i] = pymc.Dirichlet("temp_phi_%s" % i, theta=beta, trace=False)
    phi[i] = pymc.CompletedDirichlet("phi_%s" % i, phi_temp[i])

# size of document collection
M = len(df_encoded)
alpha = np.ones(K)
# theta topic distribution for each document
thetas = np.empty(M, dtype=object)
thetas_tmp = np.empty(M, dtype=object)
z = np.empty(M, dtype=object)
w = np.empty(M, dtype=object)

for m, document in enumerate(df_encoded):
    # we need to create separate variables that will be added to MCMC soup
    thetas_tmp[m] = pymc.Dirichlet("temp_theta_%s" % m, theta=alpha, trace=False)
    thetas[m] = pymc.CompletedDirichlet("theta_%s" % m, thetas_tmp[m])

for m, document in enumerate(df_encoded):
    z[m] = np.empty(len(document), dtype=object)
    w[m] = np.empty(len(document), dtype=object)
    for n, word in enumerate(document):
        # z[m][n]  is the topic for the nth word in document m
        z[m][n] = pymc.Categorical("z_%s_%s" % (m, n), p=thetas[m])
        param = pymc.Lambda("phi_z_%s_%s" % (m, n), lambda t=z[m][n], phi=phi: phi[t])
        w[m][n] = pymc.Categorical("w_%s_%s" % (m, n), p=param, value=word, observed=True)

model = pymc.Model(
    [
        pymc.Container(thetas_tmp), pymc.Container(phi_temp),
        pymc.Container(phi),
        pymc.Container(thetas),
        pymc.Container(z),
        pymc.Container(w)
    ])

mc = pymc.MCMC(model)

# when using 10K iterations model would not converge well
mc.sample(20000, 1000)

theta_result = []
for i in range(M):
    tt = mc.trace('theta_%s' % i)[:].squeeze()
    theta_result.append(tt.mean(axis=0))

theta_result = np.array(theta_result)

phi_result = []
for i in range(K):
    tt = mc.trace('phi_%s' % i)[:].squeeze()
    phi_result.append(tt.mean(axis=0))

phi_result = np.array(phi_result)

print()
print("Let's see topics ... ")


# see topics
def show_topics(top_words):
    for i in range(K):
        print("topic %s:" % i)
        print(vocab[np.argsort(phi_result[i])[::-1][:top_words]])


show_topics(2)

import math


def show_topic_term_score():
    term_score = np.empty([K, V])
    for k in range(K):
        for v in range(V):
            term_score[k][v] = phi_result[k][v] * math.log(
                phi_result[k][v] / math.pow(np.prod(phi_result[:, v]), 1 / K))
    print(term_score)
    return term_score


z_result = np.empty(M, dtype=object)
for m in range(M):
    z_result[m] = np.empty(len(df_encoded[m]))
    for n in range(len(df_encoded[m])):
        z_result[m][n] = np.bincount(mc.trace('z_%s_%s' % (m, n))[:]).argmax()

# Question: Can the topic model be used to define a topic based similarity measure between documents? (0.5)
# Answer: Yes. We can "see" a document as distribution of probabilities over topics. We can use metrics that compare discrete distributions from theta_result
# - hellinger (https://en.wikipedia.org/wiki/Hellinger_distance )
# - Kullback–Leibler divergence (which is not symetric) (https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence)
# - cosine similarity


from numpy import dot
from numpy.linalg import norm


def cos_sim(a, b):
    return dot(a, b) / (norm(a) * norm(b))


def kl(p, q):
    p = np.asarray(p, dtype=np.float)
    q = np.asarray(q, dtype=np.float)

    return np.sum(np.where(p != 0, p * np.log(p / q), 0))


def hellinger(p, q):
    return np.sqrt(np.sum((np.sqrt(p) - np.sqrt(q)) ** 2)) / np.sqrt(2)


theta_result = theta_result.squeeze()
print("Similarity Cosine, 1 - very similar")
similiarity = np.empty([M, M])
for i in range(M):
    for j in range(M):
        similiarity[i, j] = cos_sim(theta_result[i], theta_result[j])
print(similiarity.round(2))

print("Similarity hellinger, 1 - very similar")
similiarity = np.empty([M, M])
for i in range(M):
    for j in range(M):
        similiarity[i, j] = hellinger(theta_result[i], theta_result[j])
print(similiarity.round(2))

print("Similarity kl,  0 indicates that the two distributions in question are identical")
similiarity = np.empty([M, M])
for i in range(M):
    for j in range(M):
        similiarity[i, j] = kl(theta_result[i], theta_result[j])
print(similiarity.round(2))

# Question: What about a new document? How can topics be assigned to it?


# Correlated topic model

# In LDA, we assume two things:
#  - Topics in a document are independent
#  - Distribution of words in a topic is stationary

# According to "A CORRELATED TOPIC MODEL OF SCIENCE" article

# The correlated topic model. The correlated topic model (CTM) is a
# hierarchical model of document collections. The CTM models the words of
# each document from a mixture model. The mixture components are shared
# by all documents in the collection; the mixture proportions are document specific random variables.
# The CTM allows each document to exhibit multiple topics with different proportions.
#
#
# sys.exit()
#
# miu_lower = -0.01
# miu_upper = 0.01
# miu = np.empty(M, dtype=object)
# for i in range(V):
#     miu[i] = np.empty(M, dtype=object)
#     for j in range(M):
#         miu[i][j] = pymc.Uniform('miu_%s_%s' % (i, j), lower=miu_lower, upper=miu_upper)
#
# tau = pymc.Container([pymc.Wishart('tau_{0}'.format(d), n=K + 1, Tau=np.eye(K))
#                       for d in range(M)])
#
# eta = pymc.Container([pymc.MvNormal('eta_{0}'.format(d), mu=miu[d], tau=tau[d])
#                       for d in range(M)])
#
# # for m, document in enumerate(df_encoded):
# #     # we need to create separate variables that will be added to MCMC soup
# #     thetas_tmp[m] = pymc.Dirichlet("temp_theta_%s" % m, theta=alpha)
# #     thetas[m] = pymc.CompletedDirichlet("theta_%s" % m, thetas_tmp[m])
#
#
# for m, document in enumerate(df_encoded):
#     z[m] = np.empty(len(document), dtype=object)
#     w[m] = np.empty(len(document), dtype=object)
#     for n, word in enumerate(document):
#         # z[m][n]  is the topic for the nth word in document m
#         param_z = pymc.Lambda("p_z_%s_%s" % (m, n), lambda e=eta[m]: np.exp(e) / np.sum(np.exp(e)))
#         z[m][n] = pymc.Categorical("z_%s_%s" % (m, n), p=param_z)
#         param = pymc.Lambda("phi_z_%s_%s" % (m, n), lambda t=z[m][n], phi=phi: phi[t])
#         w[m][n] = pymc.Categorical("w_%s_%s" % (m, n), p=param, value=word, observed=True)
#
#
# model = pymc.Model([pymc.Container(phi), pymc.Container(miu), pymc.Container(tau), pymc.Container(eta), pymc.Container(z), pymc.Container(w)])
#
# mcmc = pymc.MCMC(model)
# mcmc.sample(20000, 5000, 1)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
{'almond': 0, 'breakfast': 1, 'butter': 2, 'cat': 3, 'dog': 4, 'eat': 5, 'enemi': 6, 'feed': 7, 'got': 8, 'like': 9, 'littl': 10, 'mortal': 11, 'mustnt': 12, 'neighbor': 13, 'peanut': 14, 'sandwich': 15, 'walnut': 16}
[array([14,  2, 15,  1]) array([ 9,  5,  0, 14, 16])
 array([13,  8, 10,  4]) array([ 3,  4, 11,  6]) array([12,  7, 14,  4])]
Dictionary size:  17
 [-----------------100%-----------------] 20000 of 20000 complete in 82.1 sec
Let's see topics ... 
topic 0:
['butter' 'enemi']
topic 1:
['walnut' 'dog']
Similarity Cosine, 1 - very similar
[[1.   0.93 0.95 0.96 1.  ]
 [0.93 1.   1.   1.   0.93]
 [0.95 1.   1.   1.   0.95]
 [0.96 1.   1.   1.   0.96]
 [1.   0.93 0.95 0.96 1.  ]]
Similarity hellinger, 1 - very similar
[[0.   0.14 0.12 0.11 0.  ]
 [0.14